<a href="https://colab.research.google.com/github/shau0000/trading-startegy-for-indian-equities-using-gradient-boosting/blob/main/gradient_boosting_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alphalens

     |████████████████████████████████| 24.0MB 123kB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
  Created wheel for alphalens: filename=alphalens-0.4.0-cp37-none-any.whl size=24027576 sha256=ae232c0f49d78eebaed8024afb604f6cc6fbf1e890dd95e9e190251ef56bb194
  Stored in directory: /root/.cache/pip/wheels/b7/fb/58/653997175d0e91a6459d84eb52e0989cf5348f0a569bd06ef5
  Created wheel for empyrical: filename=empyrical-0.5.5-cp37-none-any.whl size=39780 sha256=5df1e87ad3eb240b11034c01297b2ce57025792ef010361c93b24f734c5b89c6
  Stored in directory: /root/.cache/pip/wheels/ea/b2/c8/6769d8444d2f2e608fae2641833110668d0ffd1abeb2e9f3fc
Successfully built alphalens empyrical


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost

     |████████████████████████████████| 69.2MB 43kB/s 


In [ ]:
from pathlib import Path
import sys, os
from time import time
from tqdm import tqdm

from collections import defaultdict
from itertools import product

import numpy as np
import pandas as pd

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor

from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

from alphalens.tears import (create_summary_tear_sheet,
                             create_full_tear_sheet)

from alphalens.utils import get_clean_factor_and_forward_returns

import matplotlib.pyplot as plt
import seaborn as sns
from utils1 import MultipleTimeSeriesCV
lr = LinearRegression()

In [ ]:
year = 252
idx = pd.IndexSlice

In [ ]:
prices=pd.read_csv("/content/drive/MyDrive/QUANTL files/gradient_boosting/prices.csv")

In [ ]:
prices=prices.loc[prices['date']<'2020-01-01'].set_index(['date','ticker'])

In [ ]:
features=[]
labels=[]
for i in prices.columns:
  if ('frd') in i:
    labels.append(i)
  else:
    features.append(i)

In [ ]:
categorical_col=['year','month','weekday','Industry']

In [ ]:
train_length=[int(7*year),year]
test_length=[63,21]
lookahead=[1,5,21]
test_params=list(product(train_length,test_length,lookahead))

In [ ]:
lr_metrics=[]
for train_length,test_length,lookahead in tqdm(test_params):
  print(train_length,test_length,lookahead)
  label=f'r{lookahead:02}frd' 
  df=pd.get_dummies(prices.loc[:,features+[label]].dropna(),columns=categorical_col,drop_first=True)
  X,Y=df.drop(label,axis=1),df[label]
  n_splits=int(2*year/test_length)
  cv=MultipleTimeSeriesCV(n_splits=n_splits,test_period_length=test_length,lookahead=lookahead,train_period_length=train_length)
  ic,preds=[],[]
  for i,(train_idx,test_idx) in enumerate(cv.split(X=X)):
    X_train,Y_train=X.iloc[train_idx],Y.iloc[train_idx]
    X_test,Y_test=X.iloc[test_idx],Y.iloc[test_idx]
    lr.fit(X_train,Y_train)
    pred=lr.predict(X_test)
    preds.append(Y_test.to_frame('Y_true').assign(y_pred=pred))
    ic.append(spearmanr(Y_test,pred)[0])
  preds_df=pd.concat(preds)
  lr_metrics.append([lookahead,train_length,test_length,np.mean(ic),spearmanr(preds_df.Y_true,preds_df.y_pred)[0]])

columns=['lookahead','train_length','test_length','ic_by_day','ic']
lr_metrics=pd.DataFrame(lr_metrics,columns=columns)





  0%|          | 0/12 [00:00<?, ?it/s]

1764 63 1






  8%|▊         | 1/12 [00:37<06:53, 37.55s/it]

1764 63 5






 17%|█▋        | 2/12 [01:14<06:14, 37.46s/it]

...





 83%|████████▎ | 10/12 [08:13<01:01, 30.78s/it]

252 21 5






 92%|█████████▏| 11/12 [08:34<00:27, 27.84s/it]

252 21 21






100%|██████████| 12/12 [08:56<00:00, 44.69s/it]


In [ ]:
lr_metrics.groupby('lookahead',group_keys=False).apply(lambda x:x.nlargest(3,'ic_by_day'))

,lookahead,train_length,test_length,ic_by_day,ic
0,1,1764,63,0.066489,0.057986
9,1,252,21,0.050989,-0.000465
3,1,1764,21,0.048601,0.025675
10,5,252,21,0.105787,0.029835
1,5,1764,63,0.105157,0.081594
4,5,1764,21,0.079991,-0.003850
2,21,1764,63,0.180257,0.155919
8,21,252,63,0.129876,0.003746
5,21,1764,21,0.124327,0.111366


In [ ]:
lr_metrics.to_csv("/content/drive/MyDrive/QUANTL files/gradient_boosting/linear_regression_score.csv")


LightGBM Model Tuning¶


In [ ]:
def get_fi(model):
  fi=model.feature_importance(importance_type='gain')
  return (pd.Series(fi/fi.sum(),index=model.feature_name()))

In [ ]:
base_params = dict(boosting='gbdt',
                   objective='regression',
                   verbose=-1)

In [ ]:
max_depths = [2, 3, 5, 7]
num_leaves_opts = [2 ** i for i in max_depths]
min_data_in_leaf_opts = [250, 500, 1000]

# weight of each new tree in the ensemble
learning_rate_ops = [.01, .1, .3]

# random feature selection
feature_fraction_opts = [.3, .6, .95]

In [ ]:
param_names = ['learning_rate', 'num_leaves','feature_fraction', 'min_data_in_leaf']

In [ ]:
cv_params=list(product(learning_rate_ops ,num_leaves_opts,feature_fraction_opts,min_data_in_leaf_opts))
n_params=len(cv_params)

In [ ]:
lookaheads = [5, 21]
train_lengths = [int(7 * 252)]
test_lengths = [63]
test_params = list(product(lookaheads, train_lengths, test_lengths))
n = len(test_params)
test_param_sample = np.random.choice(list(range(n)), size=int(n), replace=False)
test_params = [test_params[i] for i in test_param_sample]

In [ ]:
categoricals = ['year', 'weekday', 'month']
for feature in categoricals:
    prices[feature] = pd.factorize(prices[feature], sort=True)[0]

In [ ]:
def ic_lgbm(preds, train_data):
    """Custom IC eval metric for lightgbm"""
    is_higher_better = True
    return 'ic', spearmanr(preds, train_data.get_label())[0], is_higher_better

In [ ]:
label_dict=dict(zip(lookaheads,labels))

In [ ]:
num_iterations = [10, 25, 50, 75] + list(range(100, 501, 50))
num_boost_round = num_iterations[-1]

In [ ]:
metric_cols = (param_names + ['daily_ic_mean', 'daily_ic_mean_n',
                              'daily_ic_median', 'daily_ic_median_n'] +
               [str(n) for n in num_iterations])

In [ ]:
for lookahead,train_length,test_length in tqdm(test_params):
  cvp=np.random.choice(list(range(n_params)),size=int(n_params/4),replace=False)
  cv_params_=[cv_params[i] for i in cvp]
  n_splits=int((2*year)/test_length)
  cv=MultipleTimeSeriesCV(n_splits=n_splits,lookahead=lookahead,train_period_length=train_length,test_period_length=test_length)
  label=label_dict[lookahead]
  outcome_data=prices.loc[:,features+[label]].dropna()
  lgb_data=lgb.Dataset(data=outcome_data.drop(label,axis=1),label=outcome_data[label],categorical_feature=categoricals,free_raw_data=False)
  predictions,metrics,feature_importance,daily_ic=[],[],[],[]
  for p,param_values in enumerate(cv_params_):
    key=f'{lookahead}-{train_length}-{test_length}-'+'-'.join([str(p) for p in param_values])
    params=dict(zip(param_names,param_values))
    params.update(base_params)
    cv_preds,n_rounds=[],[]
    ic_cv=defaultdict(list)
    for i,(train_idx,test_idx) in enumerate(cv.split(X=outcome_data)):
      lgb_train = lgb_data.subset(used_indices=train_idx.tolist(),params=params).construct()
      model=lgb.train(params=params,train_set=lgb_train,num_boost_round=num_boost_round,verbose_eval=False)
      if i==0:
        fi=get_fi(model).to_frame()
      else:
        fi[i]=get_fi(model)
      test_set=outcome_data.iloc[test_idx,:]
      X_test=test_set.loc[:,model.feature_name()]
      y_test=test_set.loc[:,label]
      y_pred={str(n): model.predict(X_test, num_iteration=n) for n in num_iterations}
      cv_preds.append(y_test.to_frame('y_test').assign(**y_pred).assign(i=i))
    cv_preds=pd.concat(cv_preds)
    by_day=cv_preds.groupby(level='date')
    ic_by_day = pd.concat([by_day.apply(lambda x: spearmanr(x.y_test, x[str(n)])[0]).to_frame(n) for n in num_iterations], axis=1)
    daily_ic_mean = ic_by_day.mean()
    daily_ic_mean_n = daily_ic_mean.idxmax()
    daily_ic_median = ic_by_day.median()
    daily_ic_median_n = daily_ic_median.idxmax()
    ic=[spearmanr(cv_preds.y_test,cv_preds[str(n)])[0] for n in num_iterations]
    metrics = pd.Series(list(param_values)+[daily_ic_mean.max(), daily_ic_mean_n, daily_ic_median.max(),daily_ic_median_n]+ic,index=metric_cols)    
    metrics.to_csv("/content/drive/MyDrive/QUANTL files/gradient_boosting/metrics"+key+".csv")
    ic_by_day.assign(**params).to_csv('/content/drive/MyDrive/QUANTL files/gradient_boosting/ic_by_day'+key+'.csv')
    fi.T.describe().T.assign(**params).to_csv('/content/drive/MyDrive/QUANTL files/gradient_boosting/fi'+key+'.csv')
    cv_preds.to_csv('/content/drive/MyDrive/QUANTL files/gradient_boosting/gradient_boosting_prediction'+key+'.csv')